<h1>Preliminaries</h1>
<p>This codebase seeks to replicate the following paper:</p>
<p><strong>Radioactive data: tracing through training</strong><br />
    <a href="https://arxiv.org/pdf/2002.00937.pdf">https://arxiv.org/pdf/2002.00937.pdf</a>
        </p>
    
<p>We have done a full refactor of the original open source Facebook code. Separately we also created a basic example demonstrating the optimization of a resnet18 cifar10 classifier. This is used as a template for the second task below.</p>
<p>This notebook goes through the following steps:</p>
   <ol>
       <li>Mark a certain subset of the CIFAR10 dataset (target data) using a resnet18 pretrained on imagenet (marking network)</li>
       <li>Use the modified CIFAR10 dataset to train a new resnet18 (target network)</li>
       <li>Attempt to detect radioactivity in the target network</li>
    </ol>
</p>
<p><strong>Note:</strong><br/>
In our example the marking network is pretrained on imagenet, while our target data is CIFAR10.
According to section 5.5 of the paper, even a marking network trained on a different
data distribution will output radioactive markings that are useful when applied to at least 10% of a class. 
This number could vary, we are just quoting the minimum radioactive data percentage shown in the paper. </p>
<p>If the marking network was trained on the same data distribution it is likely that a lower percentage of the
   target data would require marking to achieve the same p value in the detection stage.</p>
<p>When performing our testing we found that a 10% marking only produced a p-value of around 0.85, indicating that we should go higher. We are currently performing a range of tests on this.</p>

<h1>Creating Radioactive Data (Image Marking)</h1>

<p>This is self-contained within <strong>make_data_radioactive.py</strong></p>
<p>The <em>"if __name__ == '__main__':"</em> will:</p>
<ol>
    <li>Create an experiments directory <em>experiments/radioactive</em>. All further paths are relative to this directory (except tensorboard).</li>
    <li>Setup logging to <em>marking.log</em></li>
    <li>Randomly select an image class from the CIFAR10 dataset, then randomly sample a certain percentage of images in that class. After marking, these images are saved with their original index in the filename, allowing us to create a merged training set when training our classifier further on. These unmarked images are sent to TensorBoard for viewing.</li>
    <li>Download a pretrained resnet18 for use as the marking network.
    <li>Random carriers are created for all classes. These are saved to <em>carrier.pth</em> for use when detecting radioactivity.</li>
    <li>Call the main function that performs the training.</li>
</ol>
<p>Unlike a classifier, we are backpropagating all the way to the images (similar to a GAN) from a special loss function designed to maximize carrier alignment, while minimizing feature and image difference between the current and original. See section 3.3 of the paper for more details.</p>
<p>Given the above, the main method runs through batches of images, performing a random augmentation then running Adam over a desired number of epochs using this special loss function to add a subtle mark to each image</p>
<p>After each epoch we clamp all pixels so they don't drift outside of 0-255.</p>
<p>Once all epochs are completed for a given batch of images, the newly marked images are saved into <em>marked_images/{class_id}.</em></p>
<p>Each batch of images has it's own TensorBoard log for you to inspect the training loss over time.</p>
<p>It takes about 1:30 to train 500 images on a 980ti.</p>

In [5]:
%run make_data_radioactive.py

Files already downloaded and verified
INFO - 04/19/21 17:05:29 - 3 days, 21:44:40 - Randomly selected image class 9 (truck)
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - CUDA Available? True
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - Configuring marking network
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - Slicing carrier for class id 9
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - Enabling gradient on the images
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - =========== Commencing Training ===========
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - Batch Size: 32
INFO - 04/19/21 17:05:32 - 3 days, 21:44:44 - Getting original features



INFO - 04/19/21 17:05:34 - 3 days, 21:44:46 - psnr: 28.733550922357367
INFO - 04/19/21 17:05:34 - 3 days, 21:44:46 - ft_direction: 5.789703369140625
INFO - 04/19/21 17:05:34 - 3 days, 21:44:46 - ft_norm: 29.526885986328125
INFO - 04/19/21 17:05:34 - 3 days, 21:44:46 - rho: -1
INFO - 04/19/21 17:05:34 - 3 days, 21:44:46 - R: -28834.251953125
INFO - 04/19/21 17:05:35 - 3 days, 21:44:46 - Getting original features



INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - psnr: 28.762537890457438
INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - ft_direction: 5.939446926116943
INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - ft_norm: 29.720863342285156
INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - rho: -1
INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - R: -29006.5
INFO - 04/19/21 17:05:37 - 3 days, 21:44:49 - Getting original features



INFO - 04/19/21 17:05:40 - 3 days, 21:44:51 - psnr: 28.718022720300482
INFO - 04/19/21 17:05:40 - 3 days, 21:44:51 - ft_direction: 5.907112121582031
INFO - 04/19/21 17:05:40 - 3 days, 21:44:51 - ft_norm: 30.11117935180664
INFO - 04/19/21 17:05:40 - 3 days, 21:44:51 - rho: -1
INFO - 04/19/21 17:05:40 - 3 days, 21:44:51 - R: -29521.6796875
INFO - 04/19/21 17:05:40 - 3 days, 21:44:52 - Getting original features



INFO - 04/19/21 17:05:42 - 3 days, 21:44:54 - psnr: 28.75846767553024
INFO - 04/19/21 17:05:42 - 3 days, 21:44:54 - ft_direction: 6.245540618896484
INFO - 04/19/21 17:05:42 - 3 days, 21:44:54 - ft_norm: 31.68224334716797
INFO - 04/19/21 17:05:42 - 3 days, 21:44:54 - rho: -1
INFO - 04/19/21 17:05:42 - 3 days, 21:44:54 - R: -32561.783203125
INFO - 04/19/21 17:05:43 - 3 days, 21:44:55 - Getting original features


...


INFO - 04/19/21 17:06:04 - 3 days, 21:45:16 - psnr: 28.780277350329683
INFO - 04/19/21 17:06:04 - 3 days, 21:45:16 - ft_direction: 6.060686111450195
INFO - 04/19/21 17:06:04 - 3 days, 21:45:16 - ft_norm: 29.509098052978516
INFO - 04/19/21 17:06:04 - 3 days, 21:45:16 - rho: -1
INFO - 04/19/21 17:06:04 - 3 days, 21:45:16 - R: -28301.5625
INFO - 04/19/21 17:06:05 - 3 days, 21:45:17 - Getting original features



INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - psnr: 28.747992047452257
INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - ft_direction: 6.103325843811035
INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - ft_norm: 30.732141494750977
INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - rho: -1
INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - R: -30908.58984375
INFO - 04/19/21 17:06:07 - 3 days, 21:45:19 - Getting original features



INFO - 04/19/21 17:06:10 - 3 days, 21:45:21 - psnr: 28.70777850725584
INFO - 04/19/21 17:06:10 - 3 days, 21:45:21 - ft_direction: 5.6229400634765625
INFO - 04/19/21 17:06:10 - 3 days, 21:45:21 - ft_norm: 28.33430290222168
INFO - 04/19/21 17:06:10 - 3 days, 21:45:21 - rho: -1
INFO - 04/19/21 17:06:10 - 3 days, 21:45:21 - R: -26182.5625
INFO - 04/19/21 17:06:10 - 3 days, 21:45:22 - Getting original features



INFO - 04/19/21 17:06:12 - 3 days, 21:45:24 - psnr: 28.731625617825316
INFO - 04/19/21 17:06:12 - 3 days, 21:45:24 - ft_direction: 5.726820945739746
INFO - 04/19/21 17:06:12 - 3 days, 21:45:24 - ft_norm: 29.89600372314453
INFO - 04/19/21 17:06:12 - 3 days, 21:45:24 - rho: -1
INFO - 04/19/21 17:06:12 - 3 days, 21:45:24 - R: -29461.4765625
INFO - 04/19/21 17:06:13 - 3 days, 21:45:24 - Getting original features



INFO - 04/19/21 17:06:14 - 3 days, 21:45:26 - psnr: 28.680427694105436
INFO - 04/19/21 17:06:14 - 3 days, 21:45:26 - ft_direction: 5.85225772857666
INFO - 04/19/21 17:06:14 - 3 days, 21:45:26 - ft_norm: 28.58684730529785
INFO - 04/19/21 17:06:14 - 3 days, 21:45:26 - rho: -1
INFO - 04/19/21 17:06:14 - 3 days, 21:45:26 - R: -16882.9375


<p>Once this is done, inspect the TensorBoard to examine the loss for each batch and view both the original randomly selected images and their marked versions. Tensorboard logs are saved in the <em>runs</em> directory.</p>
<p>Simply run "tensorboard --logdir runs" from the repo root, and navigate to <a target="_blank" href="http://localhost:6006">http://localhost:6006</a></p>

<h1>Training A Model</h1>
<p>This is self-contained within <strong>train_marked_classifier.py</strong></p>

<p>It is pre-configured with the relative directory paths from the marking phase.</p>

<p>In this phase we take our marked CIFAR10 images from the previous step and merge them into the CIFAR10 training set using a dataloader wrapper. We then use this merged dataset to train a fresh resnet18.</p>

<p>The only difference between this and our normal resnet18 CIFAR10 example is we created a custom class <em>MergedDataset</em> that takes a standard dataset, merging dataset and a list of indexes indicating which original examples the merge dataset indexes should replace</p>

<p>The code is set to train for 60 epochs, taking about 15 minutes on a 980ti. We originally ran to 150 epochs, but overfitting starts to occur around 60. While you wait, detailed training information will show up in the TensorBoard under the <em>trained_marked_classifier</em> tag.</p>

In [2]:
%run train_marked_classifier.py

INFO - 04/15/21 21:20:46 - 1:59:57 - CUDA Available? True
Files already downloaded and verified
INFO - 04/15/21 21:20:47 - 1:59:59 - There are 500 merged examples in this dataset (1.00%)
INFO - 04/15/21 21:20:47 - 1:59:59 - Epoch Count: 60
INFO - 04/15/21 21:20:47 - 1:59:59 - No checkpoint found, starting from scratch.
INFO - 04/15/21 21:20:47 - 1:59:59 - Commence EPOCH 0



INFO - 04/15/21 21:21:05 - 2:00:17 - Saving checkpoint.
INFO - 04/15/21 21:21:05 - 2:00:17 - End of epoch 0, took 18.2987 seconds.
INFO - 04/15/21 21:21:05 - 2:00:17 - Average Train Loss: 0.002763536060228944
INFO - 04/15/21 21:21:05 - 2:00:17 - Top-1 Train Accuracy: 49.0
INFO - 04/15/21 21:21:05 - 2:00:17 - Top-1 Test Accuracy: 53.1
INFO - 04/15/21 21:21:05 - 2:00:17 - Commence EPOCH 1



INFO - 04/15/21 21:21:22 - 2:00:34 - Saving checkpoint.
INFO - 04/15/21 21:21:23 - 2:00:34 - End of epoch 1, took 17.3088 seconds.
INFO - 04/15/21 21:21:23 - 2:00:34 - Average Train Loss: 0.001959256362169981
INFO - 04/15/21 21:21:23 - 2:00:34 - Top-1 Train Accuracy: 64.486
INFO - 04/15/21 21:21:23 - 2:00:34 - Top-1 Test Accuracy: 61.69
INFO - 04/15/21 21:21:23 - 2:00:34 - Commence EPOCH 2



INFO - 04/15/21 21:21:40 - 2:00:52 - Saving checkpoint.
INFO - 04/15/21 21:21:40 - 2:00:52 - End of epoch 2, took 17.4748 seconds.
INFO - 04/15/21 21:21:40 - 2:00:52 - Average Train Loss: 0.0015526855131611228
INFO - 04/15/21 21:21:40 - 2:00:52 - Top-1 Train Accuracy: 72.062
INFO - 04/15/21 21:21:40 - 2:00:52 - Top-1 Test Accuracy: 66.1
INFO - 04/15/21 21:21:40 - 2:00:52 - Commence EPOCH 3



INFO - 04/15/21 21:21:57 - 2:01:09 - Saving checkpoint.
INFO - 04/15/21 21:21:58 - 2:01:09 - End of epoch 3, took 17.4378 seconds.
INFO - 04/15/21 21:21:58 - 2:01:09 - Average Train Loss: 0.0012737046927213669
INFO - 04/15/21 21:21:58 - 2:01:09 - Top-1 Train Accuracy: 77.066
INFO - 04/15/21 21:21:58 - 2:01:09 - Top-1 Test Accuracy: 70.13
INFO - 04/15/21 21:21:58 - 2:01:09 - Commence EPOCH 4


...


INFO - 04/15/21 21:37:12 - 2:16:24 - Saving checkpoint.
INFO - 04/15/21 21:37:12 - 2:16:24 - End of epoch 55, took 17.2071 seconds.
INFO - 04/15/21 21:37:12 - 2:16:24 - Average Train Loss: 5.465033245855011e-05
INFO - 04/15/21 21:37:12 - 2:16:24 - Top-1 Train Accuracy: 99.06
INFO - 04/15/21 21:37:12 - 2:16:24 - Top-1 Test Accuracy: 74.13
INFO - 04/15/21 21:37:12 - 2:16:24 - Commence EPOCH 56



INFO - 04/15/21 21:37:30 - 2:16:41 - Saving checkpoint.
INFO - 04/15/21 21:37:30 - 2:16:42 - End of epoch 56, took 17.5906 seconds.
INFO - 04/15/21 21:37:30 - 2:16:42 - Average Train Loss: 4.537964196060784e-05
INFO - 04/15/21 21:37:30 - 2:16:42 - Top-1 Train Accuracy: 99.208
INFO - 04/15/21 21:37:30 - 2:16:42 - Top-1 Test Accuracy: 74.11
INFO - 04/15/21 21:37:30 - 2:16:42 - Commence EPOCH 57



INFO - 04/15/21 21:37:47 - 2:16:59 - Saving checkpoint.
INFO - 04/15/21 21:37:48 - 2:16:59 - End of epoch 57, took 17.5476 seconds.
INFO - 04/15/21 21:37:48 - 2:16:59 - Average Train Loss: 5.434516424429603e-05
INFO - 04/15/21 21:37:48 - 2:16:59 - Top-1 Train Accuracy: 99.088
INFO - 04/15/21 21:37:48 - 2:16:59 - Top-1 Test Accuracy: 74.58
INFO - 04/15/21 21:37:48 - 2:16:59 - Commence EPOCH 58



INFO - 04/15/21 21:38:05 - 2:17:16 - Saving checkpoint.
INFO - 04/15/21 21:38:05 - 2:17:17 - End of epoch 58, took 17.4100 seconds.
INFO - 04/15/21 21:38:05 - 2:17:17 - Average Train Loss: 4.839987741434015e-05
INFO - 04/15/21 21:38:05 - 2:17:17 - Top-1 Train Accuracy: 99.152
INFO - 04/15/21 21:38:05 - 2:17:17 - Top-1 Test Accuracy: 74.14
INFO - 04/15/21 21:38:05 - 2:17:17 - Commence EPOCH 59



INFO - 04/15/21 21:38:22 - 2:17:34 - Saving checkpoint.
INFO - 04/15/21 21:38:22 - 2:17:34 - End of epoch 59, took 17.4339 seconds.
INFO - 04/15/21 21:38:22 - 2:17:34 - Average Train Loss: 3.888146238750778e-05
INFO - 04/15/21 21:38:22 - 2:17:34 - Top-1 Train Accuracy: 99.35
INFO - 04/15/21 21:38:22 - 2:17:34 - Top-1 Test Accuracy: 72.5


<h1>Detecting if a model is radioactive</h1>
<p>This is self-contained within <strong>detect_radioactivity.py</strong></p>
<p>Here we check for carrier alignment using cosine similarity</p>

<p>On the output, you should obtain a line with "log10(p)=...", which gives the (log of the) p-value of radioactivity detection averaged across all classes.</p>

In [3]:
%run detect_radioactivity.py

INFO - 04/15/21 21:58:30 - 2:37:42 - CUDA Available? True
INFO - 04/15/21 21:58:30 - 2:37:42 - Aligning marking and target network feature space with least squares
INFO - 04/15/21 21:58:30 - 2:37:42 - Extracting image features from marking and target networks.


/home/xl289/radioactive-lab/detect_radioactivity.py:149: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  X, residuals, rank, s = np.linalg.lstsq(features_marking, features_target)


INFO - 04/15/21 21:58:36 - 2:37:48 - Norm of residual: 3.5716e+06
INFO - 04/15/21 21:58:36 - 2:37:48 - Mean p-value is at 1 times sigma
INFO - 04/15/21 21:58:36 - 2:37:48 - Epoch of the model: 59
INFO - 04/15/21 21:58:36 - 2:37:48 - log10(p)=-1.1901726304193416
INFO - 04/15/21 21:58:36 - 2:37:48 - Total took 6.10
